In [3]:
from openmm import app, unit
from openmm.app import HBonds, NoCutoff, PDBFile
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import urllib.request  
import numpy as np
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sns
from matplotlib import colors
from IPython.display import set_matplotlib_formats
from openmm.app import PDBFile, Modeller
from openmm.app.forcefield import ForceField
import os
from pathlib import Path
from sys import stdout

In [4]:
# Define the path to your PDB file
pdb_path = r'E:\MolecularDynamicsApp\src\MolecularDynamics\data\input\A53c_mdr.pdb'

# Check if the file exists before proceeding
if not os.path.isfile(pdb_path):
    raise FileNotFoundError(f"File not found: {pdb_path}")

# Load PDB file and print topology
pdb = PDBFile(pdb_path)
print(pdb.topology)

# Create Modeller instance and load positions
modeller = Modeller(pdb.topology, pdb.positions)

# Load force field (assuming 'amber14-all.xml' is in the current working directory)
forcefield = ForceField('amber14-all.xml')

try:
    # Attempt to add hydrogens
    modeller.addHydrogens(forcefield)
except Exception as e:
    print(f"Error adding hydrogens: {str(e)}")

# Print modeller topology (optional)
print(modeller.topology)

# Write modified PDB file with hydrogens added
with open('init.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

<Topology; 2 chains, 3 residues, 56 atoms, 61 bonds>
<Topology; 2 chains, 3 residues, 92 atoms, 97 bonds>


In [5]:
pdb = PDBFile('init.pdb')
print(pdb.topology)
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml')
modeller.addHydrogens(forcefield)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, 
                nonbondedCutoff=3*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(303.15*kelvin, 1/picosecond, 2 * femtoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
potentialEnergy=True, temperature=True))
simulation.minimizeEnergy(maxIterations=1000)
state1 = simulation.context.getState(getEnergy=True)
print(state1.getPotentialEnergy())


<Topology; 2 chains, 3 residues, 92 atoms, 97 bonds>
-1145.1258544921875 kJ/mol


In [6]:
simulation.reporters = []
simulation.reporters.append(DCDReporter('traj.dcd', 1000))
simulation.reporters.append(PDBReporter("U23a_out.pdb", 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("U23a.csv", 1000, step=True, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(5000)
del simulation

#"Step","Temperature (K)","Elapsed Time (s)"
1000,278.9378059529617,0.0009999275207519531
2000,287.0895294067479,0.765683650970459
3000,291.75099386759575,1.3822999000549316
4000,324.87993230703574,1.9989221096038818
5000,311.31865953397954,2.627624034881592
